<a href="https://colab.research.google.com/github/Stravanni/Basi_di_dati/blob/main/04_SQL_sqlite_aggregazioni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduzione a SQL
@author: giovanni.simonini@unimore.it

## To run: 
- to run a cell: SHIFT + ENTER
  
## Schema Università

S (<u>Matr</u>,SNome,Citta,ACorso)

D(<u>CD</u>,CNome,Citta)

C(<u>CC</u>,CNome,CD)
- FOREIGN KEY (CD) REFERENCES D(CD)

E(<u>Matr,CC</u>,DATA,VOTO)
- FOREIGN KEY (Matr) REFERENCES S(Matr)
- FOREIGN KEY (CC) REFERENCES C(CC)
 

In [ ]:
!pip install SQLAlchemy==1.4.49
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('sqlite://', echo=False)

## Create the the tables

In [ ]:
q = '''
CREATE TABLE S (
    Matr VARCHAR(45),
    SNome VARCHAR(45),
    Citta VARCHAR(45),
    ACorso INT,
    PRIMARY KEY (Matr)
);
'''
engine.execute(q)

q = '''
CREATE TABLE D(
 CD VARCHAR(45),
 CNome VARCHAR(45),
 Citta VARCHAR(45),
 PRIMARY KEY (CD)
);
'''
engine.execute(q)

q = '''
CREATE TABLE C(
 CC VARCHAR(45),
 CNome VARCHAR(45),
 CD VARCHAR(45),
 PRIMARY KEY (CC),
 FOREIGN KEY (CD) REFERENCES D(CD)
);
'''
engine.execute(q)

q = '''
CREATE TABLE E(
 Matr VARCHAR(45),
 CC VARCHAR(45),
 DATA DATE,
 VOTO INT,
 PRIMARY KEY (Matr, CC),
 FOREIGN KEY (Matr) REFERENCES S(Matr),
 FOREIGN KEY (CC) REFERENCES C(CC)
);
'''
engine.execute(q)


q = '''
INSERT INTO S (Matr, SNome, Citta, ACorso)
VALUES
('M1','Lucia Quaranta','SA',1),
('M2','Giacomo Tedesco','PA',2),
('M3','Carla Longo','MO',1),
('M4','Ugo Rossi','MO',1),
('M5','Valeria Neri','MO',2),
('M6','Giuseppe Verdi','BO',1),
('M7','Maria Rossi',null,1);
'''
engine.execute(q)


q = '''
INSERT INTO D (CD, CNome, Citta)
VALUES
('D1','Paolo Rossi','MO'),
('D2','Maria Pastore','BO'),
('D3','Paola Caboni','FI');
'''
engine.execute(q)


q = '''
INSERT INTO C (CC,CNome, CD)
VALUES
('C1','Fisica 1','D1'),
('C2','Analisi Matematica 1','D2'),
('C3','Fisica 2','D1'),
('C4','Analisi Matematica 2','D3');
'''
engine.execute(q)


q = '''
INSERT INTO E (Matr,CC,Data,Voto)
VALUES
('M1','C1','1995-06-29',24),
('M1','C2','1996-08-09',33),
('M1','C3','1996-03-12',30),
('M2','C1','1995-06-29',28),
('M2','C2','1996-07-07',24),
('M3','C2','1996-07-07',27),
('M3','C3','1996-11-11',25),
('M4','C3','1996-11-11',33),
('M6','C2','1996-01-02',28),
('M7','C1','1995-06-29',24),
('M7','C2','1996-04-11',26),
('M7','C3','1996-06-23',27);
'''
engine.execute(q)

## Query the DB

In [ ]:
# TEMPLATE QUERY
q = '''
SELECT *
FROM S,C,D,E
'''

res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

## Funzioni di Aggregazione

In [ ]:
# 		Funzioni aggregate 
# 		(column functions)

# Dato un insieme di tuple che soddisfano un predicato
# (e.g., il risultato di una query qualsiasi di quelle
# viste finora)
# le funzione di aggregazione restituiscono un risultato
# "aggregato" calcolato su queell'insieme.

# Possono essere cosi' facilmente ritrovati:
# La media dei voti di uno studete;
# Il voto massimo in un dato esame;
# Il numero di studenti che proviene da una data citta;
# ... 

q = '''
SELECT COUNT(*)
FROM S
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
q = '''
SELECT AVG(E.VOTO)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
q = '''
SELECT MAX(E.VOTO)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
q = '''
SELECT MIN(E.VOTO)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
q = '''
SELECT SUM(E.VOTO)/COUNT(E.VOTO)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
q = '''
SELECT DISTINCT(E.CC)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
q = '''
SELECT E.CC
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
q = '''
SELECT COUNT(DISTINCT(E.CC))
FROM E
'''
# Alcuni DBMS non permettono questa sintassi
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
# Es. “Numero di studenti presenti” 
q = '''
SELECT COUNT(Matr)
FROM S
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
# Es. “Numero di studenti che hanno sostenuto almeno un esame” 
q = '''
SELECT COUNT(DISTINCT Matr)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
# Es. “Numero di studenti con anno di corso non nullo” 
q = '''
SELECT COUNT(ACorso)
FROM S
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
# Es. “Numero di anni di corso di studenti presenti” 
q = '''
SELECT COUNT(DISTINCT ACorso)
FROM S
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
# Es. “Numero di coppie distinte matricola-voto” 
q = '''
SELECT COUNT(DISTINCT Matr, Voto)
FROM E
'''
# Alcuni DBMS non permettono questa sintassi
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
# Es. “Voto medio degli esami sostenuti dalla matricola M1”
q = '''
SELECT AVG(Voto)
FROM E 
WHERE Matr='M1'
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


# che e' equivalente a
q = '''
SELECT SUM(Voto)/COUNT(Voto) 
FROM E 
WHERE Matr='M1'
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
# Es. “Studenti il cui anno di corso `e minore di quello massimo presente”
q = '''
SELECT *
FROM S
WHERE ACorso < ( SELECT MAX(ACorso)
				 FROM S)
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df


In [ ]:
# Considerate la sequente query, con funzione di aggregazione:

q = '''
SELECT Matr,MAX(Voto)
FROM E
'''
res = engine.execute(q)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df

# PERCHE' E' SBAGLIATA?

